In [2]:
import os
import tensorflow as tf
import random

In [2]:
Scaled_Path='C:\\Tensorflow\\Rescaled_DB'
hanDB_train='\\HanDB_train\\'
file_list=os.listdir(Scaled_Path+hanDB_train)

for i in range(len(file_list))
    file_path=Scaled_Path+hanDB_train+file_list[i]
    f = open(file_path,'rb')
    file_header = f.read(8)
    
    code=f.read(2)
    code_numb=int.from_bytes(code, byteorder='big')


SyntaxError: invalid syntax (<ipython-input-2-58e8e07c9c1b>, line 5)

In [4]:
def read_hgu1(file_path):
        
    f = open(file_path, 'rb')
    file_header = f.read(8)
    
    # Read Images

    # read code & check EoF
    code = f.read(2)
    width=f.read(1)[0]
    height = f.read(1)[0]
    reserved = f.read(2)
    image_byte=[]
    
    for i in range(height):
        buffer=[]
        for j in range(width):
            data=int.from_bytes(f.read(1), byteorder='big')
            buffer.append(data)
            #print(data, end=' ')
        image_byte.append(buffer)
                           
        # Reading file done
    f.close()
    
    # Read Next file
    return code, width, height, image_byte


In [9]:
Scaled_Path='C:\\Tensorflow\\Rescaled_DB'
hanDB_train='\\HanDB_train\\'
file_list=os.listdir(Scaled_Path+hanDB_train)


file_path=Scaled_Path+hanDB_train+file_list[0]
code,width,height,image_byte=read_hgu1(file_path)

print(int.from_bytes(code, byteorder='big'), width, height)
#print(image_byte)

45217 64 64


In [2]:
def make_TFRecord(file_path,TFRecord_name):
    # read files 
    file_list=os.listdir(file_path)
   
    writer = tf.python_io.TFRecordWriter("./"+TFRecord_name+'.tfrecord')
    
    for file_no in range(len(file_list)):
        # file open
        myfile=file_list[file_no]
        f = open(file_path+ myfile, 'rb')
        
        # Read Header
        file_header = f.read(8) 
        # Read Imges
    
        while True : 
        # read code & check EoF
            code = f.read(2)
            if (len(code) < 2) :
                break
            code_numb=int.from_bytes(code, byteorder='big')
            width=f.read(1)[0]
            height = f.read(1)[0]
            reserved = f.read(2)
            image_bytes=f.read(width*height)
            
            example = tf.train.Example(features=tf.train.Features(feature={
            'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[file_no])), 
            'height': tf.train.Feature(int64_list=tf.train.Int64List(value=[height])), 
            'width': tf.train.Feature(int64_list=tf.train.Int64List(value=[width])), 
            'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_bytes]))}
            ))   
            #examples.append(example) 
            writer.write(example.SerializeToString())

        f.close()
        print(str(file_no)+' '+myfile+" "+ str(code_numb)+" is added to TFRecord.")
        
    #random.shuffle(examples) 
    #for counts in range(len(examples)):
        #writer.write(examples[counts].SerializeToString())
        #examples.pop(counts)
    writer.close()

In [5]:
def test_make_TFRecord(file_path):
    # make file list
    file_list=os.listdir(file_path)
    examples=[]
    writer = tf.python_io.TFRecordWriter("./test_record.tfrecord")
    # file open
    EOF=False
    myfile=file_list[0]
    f = open(file_path+ myfile, 'rb')
    file_header = f.read(8) 
    # Read Imges
    while EOF==False : 
        # read code & check EoF
        code = f.read(2)
        if (len(code) < 2) :
            break

        code_numb=int.from_bytes(code,byteorder='big')        
        width=f.read(1)[0]
        height = f.read(1)[0]
        reserved = f.read(2)
        image_bytes=f.read(width*height)
        
        example = tf.train.Example(features=tf.train.Features(feature={'label':
        tf.train.Feature(int64_list=tf.train.Int64List(value=[code_numb])), 'height':
        tf.train.Feature(int64_list=tf.train.Int64List(value=[height])), 'width':
        tf.train.Feature(int64_list=tf.train.Int64List(value=[width])), 'image':
        tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_bytes]))}
        ))
        # Reading file done
        #print(example)
        examples.append(example) 
        break;
    f.close()
    random.shuffle(examples)    
    for counts in range(len(examples)):
        writer.write(examples[counts].SerializeToString())
    
    writer.close
        #print("File "+str(fine_no)+" is rescaled")
        # Read Next file
    print(myfile+"converted to tfrecord!!")

In [3]:
def make_TFRecord_v2(file_path,TFRecord_name):
    # read files 
    file_list=os.listdir(file_path)
    writer = tf.python_io.TFRecordWriter("./"+TFRecord_name+'.tfrecord')
    
    random.shuffle(file_list)
    
    for i in range(len(file_list)):
        # file open
        myfile=file_list[i]
        f = open(file_path+ myfile, 'rb')

        # Read Imges   
        class_id = f.read(2)
        
        class_id_int=int.from_bytes(class_id, byteorder='big')
        code = f.read(2)
        width=f.read(1)[0]
        height = f.read(1)[0]
        
        image_bytes=f.read(width*height)
            
        example = tf.train.Example(features=tf.train.Features(feature={
            'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[class_id_int])), 
            'height': tf.train.Feature(int64_list=tf.train.Int64List(value=[height])), 
            'width': tf.train.Feature(int64_list=tf.train.Int64List(value=[width])), 
            'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_bytes]))}
            ))   
            #examples.append(example) 
        writer.write(example.SerializeToString())

        f.close()
        if(i%100==0):
            print("Step :%d | is written in TFRecord" %(i))
            
    writer.close()
    print("Converting is completed")

In [5]:
origin_path='C:\\Tensorflow\\Rescaled_DB\\'
origin_whole_path= "C:\\Tensorflow\\Whole Data\\"
seri_test='SERI_Test\\'
han_train='HanDB_Train\\'
han_test='HanDB_Test\\'

pe92_train = 'PE92_Train\\'
pe92_test = 'PE92_Test\\'

file_path=origin_whole_path+pe92_test

make_TFRecord_v2(file_path,"Shuffled_PE92_Test")

#test_make_TFRecord(file_path)

Step :0 | is written in TFRecord
Step :100 | is written in TFRecord
Step :200 | is written in TFRecord
Step :300 | is written in TFRecord
Step :400 | is written in TFRecord
Step :500 | is written in TFRecord
Step :600 | is written in TFRecord
Step :700 | is written in TFRecord
Step :800 | is written in TFRecord
Step :900 | is written in TFRecord
Step :1000 | is written in TFRecord
Step :1100 | is written in TFRecord
Step :1200 | is written in TFRecord
Step :1300 | is written in TFRecord
Step :1400 | is written in TFRecord
Step :1500 | is written in TFRecord
Step :1600 | is written in TFRecord
Step :1700 | is written in TFRecord
Step :1800 | is written in TFRecord
Step :1900 | is written in TFRecord
Step :2000 | is written in TFRecord
Step :2100 | is written in TFRecord
Step :2200 | is written in TFRecord
Step :2300 | is written in TFRecord
Step :2400 | is written in TFRecord
Step :2500 | is written in TFRecord
Step :2600 | is written in TFRecord
Step :2700 | is written in TFRecord
Step

Step :22600 | is written in TFRecord
Step :22700 | is written in TFRecord
Step :22800 | is written in TFRecord
Step :22900 | is written in TFRecord
Step :23000 | is written in TFRecord
Step :23100 | is written in TFRecord
Step :23200 | is written in TFRecord
Step :23300 | is written in TFRecord
Step :23400 | is written in TFRecord
Converting is completed
